## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import sys

# Import API key
sys.path.append('../')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,cape town,ZA,-33.9258,18.4232,58.87,88,75,3.00,broken clouds
1,1,thompson,CA,55.7435,-97.8558,53.76,54,90,13.80,overcast clouds
2,2,jaisalmer,IN,26.9147,70.9181,87.66,57,100,8.90,overcast clouds
3,3,nabire,ID,-3.3667,135.4833,74.30,92,72,3.22,light rain
4,4,new norfolk,AU,-42.7826,147.0587,42.93,79,11,1.99,few clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('Please enter the minimum temperature you would prefer: '))
max_temp = float(input('Please enter the maximum temperature you would prefer: '))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
filter_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & (city_data_df['Max Temp'] <= max_temp)]
filter_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,nabire,ID,-3.3667,135.4833,74.30,92,72,3.22,light rain
11,11,jamestown,US,42.0970,-79.2353,75.02,48,1,11.01,clear sky
14,14,avarua,CK,-21.2078,-159.7750,78.85,78,20,12.66,few clouds
15,15,kamakwie,SL,9.5000,-12.2333,73.31,97,100,3.47,moderate rain
16,16,rikitea,PF,-23.1203,-134.9692,73.89,71,20,12.48,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
filter_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197 entries, 3 to 669
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              197 non-null    int64  
 1   City                 197 non-null    object 
 2   Country              197 non-null    object 
 3   Lat                  197 non-null    float64
 4   Lng                  197 non-null    float64
 5   Max Temp             197 non-null    float64
 6   Humidity             197 non-null    int64  
 7   Cloudiness           197 non-null    int64  
 8   Wind Speed           197 non-null    float64
 9   Current Description  197 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 16.9+ KB


In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = filter_df.dropna()
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 197 entries, 3 to 669
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              197 non-null    int64  
 1   City                 197 non-null    object 
 2   Country              197 non-null    object 
 3   Lat                  197 non-null    float64
 4   Lng                  197 non-null    float64
 5   Max Temp             197 non-null    float64
 6   Humidity             197 non-null    int64  
 7   Cloudiness           197 non-null    int64  
 8   Wind Speed           197 non-null    float64
 9   Current Description  197 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 16.9+ KB


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,nabire,ID,74.30,light rain,-3.3667,135.4833,
11,jamestown,US,75.02,clear sky,42.0970,-79.2353,
14,avarua,CK,78.85,few clouds,-21.2078,-159.7750,
15,kamakwie,SL,73.31,moderate rain,9.5000,-12.2333,
16,rikitea,PF,73.89,few clouds,-23.1203,-134.9692,
21,vaini,TO,75.36,light rain,-21.2000,-175.2000,
28,san quintin,MX,79.59,clear sky,30.4833,-115.9500,
29,puerto baquerizo moreno,EC,72.59,broken clouds,-0.9000,-89.6000,
30,labuhan,ID,76.59,overcast clouds,-6.8844,112.2051,
44,waingapu,ID,75.83,scattered clouds,-9.6567,120.2641,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params['location'] = f'{lat},{lng}'
    url_api = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

    # 6e. Make request and retrieve the JSON data from the search. 
    try:
       hotel = requests.get(url_api, params=params,).json()
       hotel_df.loc[index, 'Hotel Name'] = hotel['results'][0]['name']
    except:
        print(f'Hotel not found for {row["City"]}')
        continue
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
hotel_df.head()
        

Hotel not found for kamakwie
Hotel not found for rudbar
Hotel not found for albany
Hotel not found for barentu
Hotel not found for masuguru
Hotel not found for samdrup jongkhar
Hotel not found for bukama
Hotel not found for oum hadjer
Hotel not found for birao
Hotel not found for adre
Hotel not found for usevya
Hotel not found for gamba
Hotel not found for pueblo nuevo


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,nabire,ID,74.30,light rain,-3.3667,135.4833,Hotel Nusantara Nabire
11,jamestown,US,75.02,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
14,avarua,CK,78.85,few clouds,-21.2078,-159.7750,Paradise Inn
15,kamakwie,SL,73.31,moderate rain,9.5000,-12.2333,
16,rikitea,PF,73.89,few clouds,-23.1203,-134.9692,People ThankYou


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df['Hotel Name'] != ""]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,nabire,ID,74.30,light rain,-3.3667,135.4833,Hotel Nusantara Nabire
11,jamestown,US,75.02,clear sky,42.0970,-79.2353,Hampton Inn & Suites Jamestown
14,avarua,CK,78.85,few clouds,-21.2078,-159.7750,Paradise Inn
16,rikitea,PF,73.89,few clouds,-23.1203,-134.9692,People ThankYou
21,vaini,TO,75.36,light rain,-21.2000,-175.2000,Keleti Beach Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df['Max Temp']
fig = gmaps.figure(center=(20.0, 31.0), zoom_level=2.0, layout={'height': '700px', 'width': '1100px'})
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='700px', width='1100px'))